# Evaluate Base Model Endpoints using Azure AI Evaluation APIs

## Objective

This tutorial provides a step-by-step guide on how to evaluate prompts against variety of model endpoints deployed on Azure AI Platform or non Azure AI platforms. 

This guide uses Python Class as an application target which is passed to Evaluate API provided by PromptFlow SDK to evaluate results generated by LLM models against provided prompts. 

This tutorial uses the following Azure AI services:

- [azure-ai-evaluation](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/develop/evaluate-sdk)

## Time

You should expect to spend 30 minutes running this sample. 

## About this example

This example demonstrates evaluating model endpoints responses against provided prompts using azure-ai-evaluation

## Before you begin

### Installation

Install the following packages required to execute this notebook. 

In [1]:
%pip install azure-ai-evaluation
%pip install promptflow-azure
%pip install promptflow-tracing
%pip install promptflow-evals

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


### Parameters and imports

In [3]:
from pprint import pprint

import pandas as pd
import random
import os
from dotenv import load_dotenv
load_dotenv("../.credentials.env")

True

## Target Application

We will use Evaluate API provided by Prompt Flow SDK. It requires a target Application or python Function, which handles a call to LLMs and retrieve responses. 

In the notebook, we will use an Application Target `ModelEndpoints` to get answers from multiple model endpoints against provided question aka prompts. 

This application target requires list of model endpoints and their authentication keys. For simplicity, we have provided them in the `env_var` variable which is passed into init() function of `ModelEndpoints`.

In [4]:
env_var = { 
    "gpt-35-turbo": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT35_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT35_API_KEY"),
    },
    "gpt-4": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4_API_KEY"),
    },
    "gpt-4o": {
        "endpoint": os.environ.get("AZURE_OPENAI_GPT4o_ENDPOINT"),
        "key": os.environ.get("AZURE_OPENAI_GPT4o_API_KEY"),
    },
   "gpt-4o-mini" : { 
        "endpoint" : os.environ.get("AZURE_OPENAI_GPT4o-mini_ENDPOINT"), 
        "key" : os.environ.get("AZURE_OPENAI_GPT4o-mini_API_KEY"), 
    },    
}


Please provide Azure AI Project details so that traces and eval results are pushing in the project in Azure AI Studio.

In [5]:
# Initialize Azure AI project and Azure OpenAI conncetion with your environment variables
azure_ai_project = {
    "subscription_id": os.environ.get("AZURE_SUBSCRIPTION_ID"),
    "resource_group_name": os.environ.get("AZURE_RESOURCE_GROUP"),
    "project_name": os.environ.get("AZURE_PROJECT_NAME"),
}

## Model Endpoints
The following code demonstrates how to call various model endpoints, and is configured based on `env_var` set above. For any model in `env_var`, if you do not have that model deployed in your AI project, please comment it out. If you have a model that you would like to test that does not correspond with one of the types seen below, please include that type in the `__call__` function and create a helper function to call the model's endpoint via REST. 

## Data

Following code reads Json file "data.jsonl" which contains inputs to the Application Target function. It provides question, context and ground truth on each line. 

In [8]:
df = pd.read_json("../datasets/ai_data.jsonl", lines=True)
print(df.head())

                                           query  \
0         What is the capital of United Kingdom?   
1             Which tent is the most waterproof?   
2           Which camping table is the lightest?   
3  How much does TrailWalker Hiking Shoes cost?    

                                             context  \
0             United Kingdom is a country in Europe.   
1  #TrailMaster X4 Tent, price $250,## BrandOutdo...   
2  #BaseCamp Folding Table, price $60,## BrandCam...   
3  #TrailWalker Hiking Shoes, price $110## BrandT...   

                                        ground_truth  
0                                             London  
1  The TrailMaster X4 tent has a rainfly waterpro...  
2  The BaseCamp Folding Table has a weight of 15 lbs  
3    The TrailWalker Hiking Shoes are priced at $110  


## Configuration
To use Relevance and Cohenrence Evaluator, we will Azure Open AI model details as a Judge that can be passed as model config.

In [9]:
model_config = {
    "azure_endpoint": os.environ.get("AZURE_OPENAI_ENDPOINT"),
    "api_key": os.environ.get("AZURE_OPENAI_API_KEY"),
    "azure_deployment": os.environ.get("AZURE_OPENAI_DEPLOYMENT"),
    "api_version": os.environ.get("AZURE_OPENAI_API_VERSION"),
}

## Run the evaluation

The Following code runs Evaluate API and uses Content Safety, Relevance and Coherence Evaluator to evaluate results from different models.

The following are the few parameters required by Evaluate API. 

+   Data file (Prompts): It represents data file 'data.jsonl' in JSON format. Each line contains question, context and ground truth for evaluators.     

+   Application Target: It is name of python class which can route the calls to specific model endpoints using model name in conditional logic.  

+   Model Name: It is an identifier of model so that custom code in the App Target class can identify the model type and call respective LLM model using endpoint URL and auth key.  

+   Evaluators: List of evaluators is provided, to evaluate given prompts (questions) as input and output (answers) from LLM models. 

In [10]:
with open("target_api/target_api.py") as fin:
    print(fin.read())

import requests
from typing_extensions import Self
from typing import TypedDict
from promptflow.tracing import trace


class ModelEndpoints:
    def __init__(self: Self, env: dict, model_type: str) -> str:
        self.env = env
        self.model_type = model_type

    class Response(TypedDict):
        query: str
        response: str

    @trace
    def __call__(self: Self, query: str) -> Response:
        if self.model_type == "gpt-4":
            output = self.call_gpt4_endpoint(query)
        elif self.model_type == "gpt-35-turbo":
            output = self.call_gpt35_turbo_endpoint(query)
        elif self.model_type == "gpt-4o":
            output = self.call_gpt4o_endpoint(query)
        elif self.model_type == "gpt-4o-mini":
            output = self.call_gpt4o_mini_endpoint(query)
        else:
            output = self.call_default_endpoint(query)

        return output

    def query(self: Self, endpoint: str, headers: str, payload: str) -> str:
        response = requests

In [11]:
from target_api.target_api import ModelEndpoints

In [21]:
import pathlib
import time
from target_api.target_api import ModelEndpoints
from azure.ai.evaluation import evaluate
from azure.ai.evaluation import (
    RelevanceEvaluator,
    CoherenceEvaluator,
    GroundednessEvaluator,
    FluencyEvaluator,
    SimilarityEvaluator,
)

relevance_evaluator = RelevanceEvaluator(model_config)
coherence_evaluator = CoherenceEvaluator(model_config)
groundedness_evaluator = GroundednessEvaluator(model_config)
fluency_evaluator = FluencyEvaluator(model_config)
similarity_evaluator = SimilarityEvaluator(model_config)


models = ["gpt-35-turbo","gpt-4","gpt-4o","gpt-4o-mini"]

path = str(pathlib.Path(pathlib.Path.cwd())) + "/ai_data.jsonl"

for model in models:
    print(" Evaluating AI-assisted metrics - ", model)
    print("-----------------------------------")
    randomNum = random.randint(1111, 9999)
    results = evaluate(
        azure_ai_project=azure_ai_project, 
        evaluation_name="Eval_" + model.title() + "_Run-" + str(randomNum),
        data=path,
        target=ModelEndpoints(env_var, model),
        
        evaluators={
            "relevance": relevance_evaluator,
            "groundedness": groundedness_evaluator,
            "coherence": coherence_evaluator,
            "fluency": fluency_evaluator,
            "similarity": similarity_evaluator,

        },
        evaluator_config={

            "relevance": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
                },

            "groundedness": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "coherence": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "fluency": {
                "column_mapping": {
                    "query": "${data.query}", 
                    "context": "${data.context}", 
                    "response": "${target.response}"}
            },

            "similarity": {
                "column_mapping": {
                    "ground_truth": "${data.ground_truth}",
                    "context": "${data.context}", 
                    "query": "${data.query}",
                    "response": "${target.response}"}
            },
        },
    )
    #time.sleep(60) ## To avoid rate limiting throttling


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=target_api_target_api_modelendpoints_wgj7ftie_20241220_100513_110685


[2024-12-20 10:05:16 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run target_api_target_api_modelendpoints_wgj7ftie_20241220_100513_110685, log path: C:\Users\sumohammed\.promptflow\.runs\target_api_target_api_modelendpoints_wgj7ftie_20241220_100513_110685\logs.txt


Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_b1sh_ht9_20241220_100525_255249
Prompt flow service has started...
Prompt flow service has started...


[2024-12-20 10:05:25 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:05:25 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:05:25 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:05:25 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_b1sh_ht9_20241220_100525_255249, log path: C:\Users\sumohammed\.promptflow\.runs\azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_b1sh_ht9_20241220_100525_255249\logs.txt
[2024-12-20 10:05:25 +0000][promptflow._core.entry_met

You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_guo2pi8w_20241220_100525_257149
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_y1bymk0s_20241220_100525_257149
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_1q9mvwwu_20241220_100525_258150
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_similarity_similarity_asyncsimilarityevaluator_oqnlpusf_20241220_100525_256150
2024-12-20 10:05:29 +0000   29880 execution.bulk     INFO     Finished 1 / 4 lines.
2024-12-20 10:05:29 +0000   29880 execution.bulk     INFO     Average execution time for completed lines: 4.15 

...

Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_upbieylj_20241220_100834_017613
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_ub3fnmuu_20241220_100834_015613
Prompt flow service has started...
Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_4a7xwv2_20241220_100834_016613
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_rra6xhv9_20241220_100834_018612


[2024-12-20 10:08:34 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:08:34 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:08:34 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:08:34 +0000][promptflow._core.entry_meta_generator][WARNING] - Generate meta in current process and timeout won't take effect. Please handle timeout manually outside current process.
[2024-12-20 10:08:34 +0000][promptflow._sdk._orchestrator.run_submitter][INFO] - Submitting run azure_ai_evaluation_evaluators_common_base_eval_asyncevaluatorbase_upbieylj_20241220_100834_017613, log path: C:

Prompt flow service has started...
You can view the traces in local from http://127.0.0.1:23333/v1.0/ui/traces/?#run=azure_ai_evaluation_evaluators_similarity_similarity_asyncsimilarityevaluator_qtfg4ud9_20241220_100834_018612


[2024-12-20 10:08:34 +0000][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 8 seconds. Please contact Azure support service if you would like to further increase the default rate limit.'}}
[2024-12-20 10:08:34 +0000][promptflow.core._prompty_utils][WARNING] - RateLimitError #0, Retry-After=8, Back off 8.0 seconds for retry.
[2024-12-20 10:08:34 +0000][promptflow.core._prompty_utils][ERROR] - Exception occurs: RateLimitError: Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-08-01-preview have exceeded token rate limit of your current AIServices S0 pricing tier. Please retry after 8 seconds. Please contact Azure support servic

2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Finished 1 / 4 lines.
2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Average execution time for completed lines: 2.88 seconds. Estimated time for incomplete lines: 8.64 seconds.
2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Finished 1 / 4 lines.
2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Average execution time for completed lines: 3.29 seconds. Estimated time for incomplete lines: 9.87 seconds.
2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Finished 1 / 4 lines.
2024-12-20 10:08:37 +0000   29880 execution.bulk     INFO     Average execution time for completed lines: 3.51 seconds. Estimated time for incomplete lines: 10.53 seconds.
2024-12-20 10:08:38 +0000   29880 execution.bulk     INFO     Finished 1 / 4 lines.
2024-12-20 10:08:38 +0000   29880 execution.bulk     INFO     Average execution time for completed lines: 4.4 seconds. Estimated time for incomplete l

View the results

In [22]:
pd.DataFrame(results["rows"])

,outputs.query,outputs.response,inputs.query,inputs.context,inputs.ground_truth,outputs.relevance.relevance,outputs.relevance.gpt_relevance,outputs.relevance.relevance_reason,outputs.groundedness.groundedness,outputs.groundedness.gpt_groundedness,outputs.groundedness.groundedness_reason,outputs.coherence.coherence,outputs.coherence.gpt_coherence,outputs.coherence.coherence_reason,outputs.fluency.fluency,outputs.fluency.gpt_fluency,outputs.fluency.fluency_reason,outputs.similarity.similarity,outputs.similarity.gpt_similarity,line_number
0,What is the capital of United Kingdom?,The capital of the United Kingdom is London.,What is the capital of United Kingdom?,United Kingdom is a country in Europe.,London,4,4,"The response is accurate and complete, directl...",3,3,The RESPONSE introduces information not presen...,4,4,The response is coherent because it directly a...,3,3,"The response is clear, grammatically correct, ...",5,5,0
1,Which tent is the most waterproof?,"When looking for the most waterproof tent, con...",Which tent is the most waterproof?,"#TrailMaster X4 Tent, price $250,## BrandOutdo...",The TrailMaster X4 tent has a rainfly waterpro...,4,4,The RESPONSE fully addresses the QUERY by expl...,1,1,The RESPONSE is entirely unrelated to the CONT...,4,4,The RESPONSE is coherent and effectively addre...,4,4,"The RESPONSE is well-articulated, with good co...",2,2,1
2,Which camping table is the lightest?,As of my last knowledge update in October 2023...,Which camping table is the lightest?,"#BaseCamp Folding Table, price $60,## BrandCam...",The BaseCamp Folding Table has a weight of 15 lbs,5,5,The response fully addresses the query by prov...,1,1,The RESPONSE is entirely unrelated to the CONT...,4,4,"The RESPONSE is coherent, directly addressing ...",4,4,"The RESPONSE is well-articulated, with good co...",1,1,2
3,How much does TrailWalker Hiking Shoes cost?,The price of TrailWalker hiking shoes can vary...,How much does TrailWalker Hiking Shoes cost?,"#TrailWalker Hiking Shoes, price $110## BrandT...",The TrailWalker Hiking Shoes are priced at $110,3,3,The response provides a general price range an...,1,1,The RESPONSE is ungrounded as it does not refl...,4,4,The response is coherent and effectively addre...,4,4,The RESPONSE is well-articulated with good con...,1,1,3
